In [1]:
import torch 
import torchvision
import numpy as np

from PIL import Image
import os
import matplotlib.pyplot as plt
import math




In [2]:
import create_dataset
import torchvision
import importlib


data_small=create_dataset.DatasetFromSeparatePatches("dataset/",number_of_patch_side=1)
#data_small.show(20000)

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'dataset/'

In [2]:
import create_dataset

create_dataset= importlib.reload(create_dataset)

dataset=create_dataset.DatasetFromSeparatePatches("dataset/",number_of_patch_side=5)
#dataset.show(2000)

NameError: name 'importlib' is not defined

## first method : Virtual restoration of cracks in digitized image of paintings

Virtual restoration of cracks in digitized image of paintings :doi:10.1088/1742-6596/249/1/012059


Use morphological operation (Opening Top-Hat/ closing top of the hat) to restore images

## other papers :
User Intervention Based Detection & Removal of Cracks from Digitized Paintings
DOI:10.1109/ICSIP.2014.7 (nice bibliography & dataset)



## experiences
 



the operation are based on mathematical morphology <br />
scipy provide a implementation that we can use. <br />
mathématical morphology is base on the application of structuring elements on the image <br/>



first step : Identify the elements that ressamble cracks : we use OTH (opening top of the hat operators)
two parameters to fix: 
- the shape/sizeof the structuring element : according to the article a disk shaped is a good shape. we need to fix the size of the structuring element<br />
- the tresholding limit 

the article also said that we can apply the dilatation/erosion n times. This procedure is equivalent to apply a structuring element nB where nB is the structuring element obtain by applying n times the dilatation to B. the only importance parameter is thus the shape and "size" of the original structuring element. ("size" because I think there is a definition that I don't understand)




In [1]:
from skimage.morphology import disk
from skimage.morphology import black_tophat,white_tophat
import matplotlib.pyplot as plt
import numpy as np

# test the effect of the "size" of the structuring element
size=2

image=data[2000]


luminosity=image.sum(axis=-1)/3


result=np.stack([black_tophat(luminosity,selem=disk(size)) for i in range(3)],axis=-1)

result_normalize=(result-result.min())/(result.max()-result.min())

plt.figure(figsize=(10,10))
#plt.imshow(result_normalize)

size=5
result=np.stack([black_tophat(luminosity,selem=disk(size)) for i in range(3)],axis=-1)
result_normalize=(result-result.min())/(result.max()-result.min())

plt.figure(figsize=(10,10))
#plt.imshow(result_normalize)


NameError: name 'data' is not defined

In [3]:
def get_transformed_image(luminosity,size):
    return black_tophat(luminosity,selem=disk(size))

def get_mask(luminosity,size,treshold):

    return (get_transformed_image(luminosity,size)>treshold)*1.

#visualitation of the effect of the tresholding

list_image=[get_mask(luminosity,3,i) for i in [3,4,6,12,16,18,20,25]]


nombre_image=len(list_image)
size_square=np.ceil(np.sqrt(nombre_image)).astype(np.int64)

plt.figure(figsize=(20,20)) # specifying the overall grid size

for i in range(nombre_image):
    plt.subplot(size_square,size_square,i+1)    # the number of images in the grid is 5*5 (25)
    #plt.imshow(list_image[i])


NameError: name 'luminosity' is not defined

In [123]:

from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
# dd/mm/YY H:M:S
dt_string = now.strftime("%d_%m_%Y %Hh %Mm %Ss")

save_directory="logs/top_of_the_hat_test/"+dt_string+"/"
assert not(os.path.isdir(save_directory))

list_image=[]
list_parameters=[]
treshold_values=[3,4,6,12,16,18,20,25,30,50]
size_values= [1,2,4,6,9,15]

for treshold in treshold_values:

    list_size_array=[]
    for size in size_values:
        list_size_array.append(get_mask(luminosity,size,treshold))
        list_parameters.append([size,treshold])
    list_image.append(list_size_array)

array_img=np.array(list_image)
array_img.shape

(10, 6, 1280, 1280)

In [4]:
import plotly.express as px
from skimage import io
from plotly.offline import init_notebook_mode
import plotly.io as pio
pio.renderers.default = "notebook_connected"

fig = px.imshow(255*array_img, animation_frame=0,facet_col=1, binary_string=True,binary_compression_level=0)
#fig.show()

NameError: name 'array_img' is not defined

In [5]:
#plt.imshow(array_img[0][0])

In [135]:
from skimage.filters import threshold_otsu
#optimum value seems to be 4-6 for size and ~19 for threshold
values=get_transformed_image(luminosity,5)

tresh_otsu=threshold_otsu(values)

C:\Users\lavra\miniconda3\envs\torch\lib\site-packages\skimage\morphology\misc.py:39: FutureWarning:

`selem` is a deprecated argument name for `black_tophat`. It will be removed in version 1.0.Please use `footprint` instead.



30.517578125

### results

In [6]:

def plot_exemple_patch(index):
    image=dataset[index]
    size=9
    treshold=30

    plt.figure(figsize=(40,100))

    plt.subplot(1,2,1)
    plt.imshow(image)
    ax1=plt.subplot(1,2,2)

    luminosity=image.mean(axis=-1)
    plt.imshow(get_mask(luminosity,size,treshold))
    plt.show()


for i in [200,400,1000,2000,3000]:
    #plot_exemple_patch(i)

IndentationError: expected an indented block (Temp/ipykernel_18888/3862253483.py, line 18)

# conclusion :

The classical morphological method works well for certain cracks, but in some situations, it's difficult to detect anithing. Somethimes some portion of the images are missclassified. based on color or shape, it's somethimes possible to separate cracks from the rest of the paint (i've not implemented thoses methods).  <br />
